<img src="../../images/banners/python-advanced.png" width="600"/>

# <img src="../../images/logos/python.png" width="23"/> What Is the Python Global Interpreter Lock (GIL)? 


<a class="anchor" id="table_of_contents"></a>
## <img src="../../images/logos/toc.png" width="20"/> Table of Contents
* [What Problem Did the GIL Solve for Python?](#what_problem_did_the_gil_solve_for_python?)
* [Why Was the GIL Chosen as the Solution?](#why_was_the_gil_chosen_as_the_solution?)
* [The Impact on Multi-Threaded Python Programs](#the_impact_on_multi-threaded_python_programs)
* [Why Hasn’t the GIL Been Removed Yet?](#why_hasn’t_the_gil_been_removed_yet?)
* [Why Wasn’t It Removed in Python 3?](#why_wasn’t_it_removed_in_python_3?)
* [How to Deal With Python’s GIL](#how_to_deal_with_python’s_gil)

---

<p>The Python Global Interpreter Lock or <a href="https://wiki.python.org/moin/GlobalInterpreterLock">GIL</a>, in simple words, is a mutex (or a lock) that allows only one <a href="https://realpython.com/intro-to-python-threading/">thread</a> to hold the control of the Python interpreter. </p>

<p>This means that only one thread can be in a state of execution at any point in time. The impact of the GIL isn’t visible to developers who execute single-threaded programs, but it can be a performance bottleneck in CPU-bound and multi-threaded code.</p>

<p>Since the GIL allows only one thread to execute at a time even in a multi-threaded architecture with more than one CPU core, the GIL has gained a reputation as an “infamous” feature of Python.</p>

<p><strong>In this article you’ll learn how the GIL affects the performance of your Python programs, and how you can mitigate the impact it might have on your code.</strong></p>

<a class="anchor" id="what_problem_did_the_gil_solve_for_python?"></a>
## What Problem Did the GIL Solve for Python? [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Python uses reference counting for <a href="https://realpython.com/python-memory-management/">memory management</a>. It means that objects created in Python have a reference count variable that keeps track of the number of references that point to the object. When this count reaches zero, the memory occupied by the object is released.</p>

<p>Let’s take a look at a brief code example to demonstrate how reference counting works:</p>

In [ ]:
>>>>>> import sys
>>> a = []
>>> b = a
>>> sys.getrefcount(a)
3

<p>In the above example, the reference count for the empty list object <code>[]</code> was 3. The list object was referenced by <code>a</code>, <code>b</code> and the argument passed to <code>sys.getrefcount()</code>.</p>

<p>Back to the GIL:</p>

<p>The problem was that this reference count variable needed protection from race conditions where two threads increase or decrease its value simultaneously. If this happens, it can cause either leaked memory that is never released or, even worse, incorrectly release the memory while a reference to that object still exists. This can cause crashes or other “weird” bugs in your Python programs.</p>

<p>This reference count variable can be kept safe by adding <em>locks</em> to all data structures that are shared across threads so that they are not modified inconsistently. </p>

<p>But adding a lock to each object or groups of objects means multiple locks will exist which can cause another problem—Deadlocks (deadlocks can only happen if there is more than one lock). Another side effect would be decreased performance caused by the repeated acquisition and release of locks.</p>

<p>The GIL is a single lock on the interpreter itself which adds a rule that execution of any Python bytecode requires acquiring the interpreter lock. This prevents deadlocks (as there is only one lock) and doesn’t introduce much performance overhead. But it effectively makes any CPU-bound Python program single-threaded.</p>

<p>The GIL, although used by interpreters for other languages like Ruby, is not the only solution to this problem. Some languages avoid the requirement of a GIL for thread-safe memory management by using approaches other than reference counting, such as garbage collection. </p>

<p>On the other hand, this means that those languages often have to compensate for the loss of single threaded performance benefits of a GIL by adding other performance boosting features like JIT compilers.</p>

<a class="anchor" id="why_was_the_gil_chosen_as_the_solution?"></a>
## Why Was the GIL Chosen as the Solution? [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>So, why was an approach that is seemingly so obstructing used in Python? Was it a bad decision by the developers of Python? </p>

<p>Well, in the <a href="https://youtu.be/KVKufdTphKs?t=12m11s">words of Larry Hastings</a>, the design decision of the GIL is one of the things that made Python as popular as it is today.</p>

<p>Python has been around since the days when operating systems did not have a concept of threads. Python was designed to be easy-to-use in order to make development quicker and more and more developers started using it. </p>

<p>A lot of extensions were being written for the existing C libraries whose features were needed in Python. To prevent inconsistent changes, these C extensions required a thread-safe memory management which the GIL provided.</p>

<p>The GIL is simple to implement and was easily added to Python. It provides a performance increase to single-threaded programs as only one lock needs to be managed.</p>

<p>C libraries that were not thread-safe became easier to integrate. And these C extensions became one of the reasons why Python was readily adopted by different communities.</p>

<p>As you can see, the GIL was a pragmatic solution to a difficult problem that the <a href="https://realpython.com/cpython-source-code-guide/">CPython</a> developers faced early on in Python’s life.</p>

<a class="anchor" id="the_impact_on_multi-threaded_python_programs"></a>
## The Impact on Multi-Threaded Python Programs [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>When you look at a typical Python program—or any computer program for that matter—there’s a difference between those that are CPU-bound in their performance and those that are I/O-bound.</p>

<p>CPU-bound programs are those that are pushing the CPU to its limit. This includes programs that do mathematical computations like matrix multiplications, searching, image processing, etc. </p>

<p>I/O-bound programs are the ones that spend time waiting for <a href="https://realpython.com/python-input-output/">Input/Output</a> which can come from a user, file, database, network, etc. I/O-bound programs sometimes have to wait for a significant amount of time till they get what they need from the source due to the fact that the source may need to do its own processing before the input/output is ready, for example, a user thinking about what to enter into an input prompt or a database query running in its own process.</p>

<p>Let’s have a look at a simple CPU-bound program that performs a countdown:</p>

In [ ]:
# single_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

<p>Running this code on my system with 4 cores gave the following output:</p>

```sh
$ python single_threaded.py
Time taken in seconds - 6.20024037361145
```

<p>Now I modified the code a bit to do to the same countdown using two threads in parallel:</p>

In [ ]:
# multi_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time taken in seconds -', end - start)

<p>And when I ran it again:</p>

```sh
$ python multi_threaded.py
Time taken in seconds - 6.924342632293701
```

<p>As you can see, both versions take almost same amount of time to finish. In the multi-threaded version the GIL prevented the CPU-bound threads from executing in parellel.</p>

<p>The GIL does not have much impact on the performance of I/O-bound multi-threaded programs as the lock is shared between threads while they are waiting for I/O. </p>

<p>But a program whose threads are entirely CPU-bound, e.g., a program that processes an image in parts using threads, would not only become single threaded due to the lock but will also see an increase in execution time, as seen in the above example, in comparison to a scenario where it was written to be entirely single-threaded. </p>

<p>This increase is the result of acquire and release overheads added by the lock.</p>

<a class="anchor" id="why_hasn’t_the_gil_been_removed_yet?"></a>
## Why Hasn’t the GIL Been Removed Yet? [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>The developers of Python receive a lot of complaints regarding this but a language as popular as Python cannot bring a change as significant as the removal of GIL without causing backward incompatibility issues. </p>

<p>The GIL can obviously be removed and this has been done multiple times in the past by the developers and researchers but all those attempts broke the existing C extensions which depend heavily on the solution that the GIL provides.</p>

<p>Of course, there are other solutions to the problem that the GIL solves but some of them decrease the performance of single-threaded and multi-threaded I/O-bound programs and some of them are just too difficult. After all, you wouldn’t want your existing Python programs to run slower after a new version comes out, right?</p>

<p>The creator and BDFL of Python, Guido van Rossum, gave an answer to the community in September 2007 in his article <a href="https://www.artima.com/weblogs/viewpost.jsp?thread=214235">“It isn’t Easy to remove the GIL”</a>:</p>

<p>And this condition hasn’t been fulfilled by any of the attempts made since.</p>

<a class="anchor" id="why_wasn’t_it_removed_in_python_3?"></a>
## Why Wasn’t It Removed in Python 3? [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Python 3 did have a chance to start a lot of features from scratch and in the process, broke some of the existing C extensions which then required changes to be updated and ported to work with Python 3. This was the reason why the early versions of Python 3 saw slower adoption by the community. </p>

<p>But why wasn’t GIL removed alongside? </p>

<p>Removing the GIL would have made Python 3 slower in comparison to Python 2 in single-threaded performance and you can imagine what that would have resulted in. You can’t argue with the single-threaded performance benefits of the GIL. So the result is that Python 3 still has the GIL.</p>

<p>But Python 3 did bring a major improvement to the existing GIL—</p>

<p>We discussed the impact of GIL on “only CPU-bound” and “only I/O-bound” multi-threaded programs but what about the programs where some threads are I/O-bound and some are CPU-bound? </p>

<p>In such programs, Python’s GIL was known to starve the I/O-bound threads by not giving them a chance to acquire the GIL from CPU-bound threads.</p>

<p>This was because of a mechanism built into Python that forced threads to release the GIL <strong>after a fixed interval</strong> of continuous use and if nobody else acquired the GIL, the same thread could continue its use.</p>

In [ ]:
>>>>>> import sys
>>> # The interval is set to 100 instructions:
>>> sys.getcheckinterval()
100

<p>The problem in this mechanism was that most of the time the CPU-bound thread would reacquire the GIL itself before other threads could acquire it. This was researched by David Beazley and visualizations can be found <a href="http://www.dabeaz.com/blog/2010/01/python-gil-visualized.html">here</a>.</p>

<p>This problem was fixed in Python 3.2 in 2009 by Antoine Pitrou who <a href="https://mail.python.org/pipermail/python-dev/2009-October/093321.html">added a mechanism</a> of looking at the number of GIL acquisition requests by other threads that got dropped and not allowing the current thread to reacquire GIL before other threads got a chance to run.</p>

<a class="anchor" id="how_to_deal_with_python’s_gil"></a>
## How to Deal With Python’s GIL [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>If the GIL is causing you problems, here a few approaches you can try:</p>

<p><strong>Multi-processing vs multi-threading:</strong> The most popular way is to use a multi-processing approach where you use multiple processes instead of threads. Each Python process gets its own Python interpreter and memory space so the GIL won’t be a problem. Python has a <a href="https://docs.python.org/2/library/multiprocessing.html"><code>multiprocessing</code></a> module which lets us create processes easily like this:</p>

In [ ]:
from multiprocessing import Pool
import time

COUNT = 50000000
def countdown(n):
    while n>0:
        n -= 1

if __name__ == '__main__':
    pool = Pool(processes=2)
    start = time.time()
    r1 = pool.apply_async(countdown, [COUNT//2])
    r2 = pool.apply_async(countdown, [COUNT//2])
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)

<p>Running this on my system gave this output:</p>

```sh
$ python multiprocess.py
Time taken in seconds - 4.060242414474487
```

<p>A decent performance increase compared to the multi-threaded version, right? </p>

<p>The time didn’t drop to half of what we saw above because process management has its own overheads. Multiple processes are heavier than multiple threads, so, keep in mind that this could become a scaling bottleneck.</p>

<p><strong>Alternative Python interpreters:</strong> Python has multiple interpreter implementations. CPython, Jython, IronPython and <a href="https://realpython.com/pypy-faster-python/">PyPy</a>, written in <a href="https://realpython.com/c-for-python-programmers/">C</a>, <a href="https://realpython.com/oop-in-python-vs-java/">Java</a>, C# and Python respectively, are the most popular ones. GIL exists only in the original Python implementation that is CPython. If your program, with its libraries, is available for one of the other implementations then you can try them out as well.</p>

<p><strong>Just wait it out:</strong> While many Python users take advantage of the single-threaded performance benefits of GIL. The multi-threading programmers don’t have to fret as some of the brightest minds in the Python community are working to remove the GIL from CPython. One such attempt is known as the <a href="https://github.com/larryhastings/gilectomy">Gilectomy</a>. </p>

<p>The Python GIL is often regarded as a mysterious and difficult topic. But keep in mind that as a Pythonista you’re usually only affected by it if you are writing C extensions or if you’re using CPU-bound multi-threading in your programs.</p>

<p>In that case, this article should give you everything you need to understand what the GIL is and how to deal with it in your own projects. And if you want to understand the low-level inner workings of GIL, I’d recommend you watch the <a href="https://youtu.be/Obt-vMVdM8s">Understanding the Python GIL</a> talk by David Beazley.</p>